# Chat con Video de YouTube: Nuestro Primer Sistema RAG 

En este taller construiremos una **Inteligencia Artificial que puede "ver" un video de YouTube y responder preguntas sobre él**.

Usaremos una técnica llamada **RAG (Retrieval-Augmented Generation)**.

### Tecnologías que usaremos:
- **Google Gemini**: El "cerebro" que genera las respuestas y crea los embeddings.
- **ChromaDB**: Nuestra base de datos vectorial (memoria rápida).
- **YouTube Transcript API**: Para leer los subtítulos del video.

## 1. Instalación de Librerías
Primero, necesitamos instalar las herramientas necesarias en nuestro entorno.

In [ ]:
# Instalamos las librerías necesarias
%pip install -q -U google-generativeai chromadb youtube-transcript-api

## 2. Configuración Inicial
Importamos las librerías y configuramos nuestra llave de acceso (API Key) para Google Gemini.

Puedes obtener tu API Key en: https://aistudio.google.com/api-keys

In [ ]:
import google.generativeai as genai
import chromadb
from youtube_transcript_api import YouTubeTranscriptApi
import time

# REEMPLAZA ESTO CON TU API KEY DE GOOGLE
GOOGLE_API_KEY = "TU API KEY"

genai.configure(api_key=GOOGLE_API_KEY)

## 3. Ingesta de Datos (Creando la Base de Conocimiento)
Para que la IA sepa sobre el video, primero necesitamos extraer el texto (transcripción).

In [ ]:
def extract_transcript(youtube_url):
    """Obtiene la lista de objetos de transcripción (texto + tiempo) en lugar de solo texto plano"""
    try:
        if "v=" in youtube_url:
            video_id = youtube_url.split("v=")[1].split("&")[0]
        else:
            video_id = youtube_url

        ytt_api = YouTubeTranscriptApi()
        transcript_list_obj = ytt_api.list(video_id)
        transcript = transcript_list_obj.find_transcript(['es', 'en'])
        return transcript.fetch() # Devolvemos la lista completa con tiempos

    except Exception as e:
        print(f"Error: {e}")
        return None

VIDEO_URL = "https://www.youtube.com/watch?v=N_pbqiocSAE"
video_data = extract_transcript(VIDEO_URL)

if video_data:
    print("Transcripción extraída con éxito!")
    print(f"Total de frases: {len(video_data)}")
    print(f"Ejemplo: {video_data[0].text} (Inicio: {video_data[0].start}s)")
else:
    print("No se pudo extraer la transcripción.")

## 4. Chunking (Dividir para Conquistar)
Los modelos de IA tienen un límite de cuánto texto pueden leer a la vez. Además, para buscar información específica, **es mejor tener fragmentos pequeños y enfocados**.

In [ ]:
def create_smart_chunks(raw_transcript, chunk_size=1000):
    """Agrupa texto preservando el Timestamp de inicio"""
    chunks = []
    current_chunk_text = ""
    current_start_time = 0.0
    
    if raw_transcript:
        current_start_time = raw_transcript[0].start

    for item in raw_transcript:
        text = item.text
        # Si iniciamos grupo nuevo, guardamos tiempo
        if current_chunk_text == "":
            current_start_time = item.start
            
        current_chunk_text += " " + text
        
        if len(current_chunk_text) >= chunk_size:
            # Formato MM:SS
            minutes = int(current_start_time // 60)
            seconds = int(current_start_time % 60)
            time_str = f"{minutes:02d}:{seconds:02d}"
            
            chunks.append({"text": current_chunk_text.strip(), "timestamp": time_str})
            current_chunk_text = ""
            
    # Último chunk
    if current_chunk_text:
        minutes = int(current_start_time // 60)
        seconds = int(current_start_time % 60)
        chunks.append({"text": current_chunk_text.strip(), "timestamp": f"{minutes:02d}:{seconds:02d}"})
        
    return chunks

# Dividimos el texto inteligentemente
chunks = create_smart_chunks(video_data)

print(f"Texto dividido en {len(chunks)} fragmentos con metadata.")
print(f"Ejemplo Chunk 1 [{chunks[0]['timestamp']}]: {chunks[0]['text'][:100]}...")

## 5. Embeddings y Base de Datos Vectorial (ChromaDB)
Aquí ocurre la magia. Convertiremos cada "chunk" de texto en una lista de números llamada **Embedding**. Los embeddings representan el **significado** del texto.

Guardaremos estos embeddings en **ChromaDB** para poder buscar rápidamente después.

In [ ]:
# 1. Inicializamos ChromaDB
chroma_client = chromadb.Client()
collection_name = "youtube_rag_data"
try:
    chroma_client.delete_collection(name=collection_name)
except:
    pass
collection = chroma_client.create_collection(name=collection_name)

print("Generando embeddings y guardando...")

documents = []
metadatas = []
ids = []
embeddings = []

for i, chunk in enumerate(chunks):
    # Generamos el embedding
    response = genai.embed_content(
        model="models/text-embedding-004",
        content=chunk['text'],
        task_type="retrieval_document"
    )
    
    # Guardamos Texto Y Metadata (Tiempo)
    documents.append(chunk['text'])
    metadatas.append({"timestamp": chunk['timestamp']})
    ids.append(str(i))
    embeddings.append(response['embedding'])

collection.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

print("¡Base de conocimiento lista!")

## 6. El Cerebro
Ahora uniremos todo.

Cuando hagas una pregunta:
1. **Búsqueda**: Se convierte tu pregunta en números y buscaremos los chunks más parecidos en ChromaDB.
2. **Contexto**: Tomaremos esos chunks y se los mostraremos a la IA.
3. **Respuesta**: La IA responderá usando *sólo* esa información.

In [ ]:
# Configuramos modelo (Usamos Flash por estabilidad)
model = genai.GenerativeModel('models/gemini-2.5-flash')
chat_history = []

def generate_answer(system_prompt):
    """INTENTO DE GENERACIÓN CON RETRY"""
    try:
        return model.generate_content(system_prompt).text
    except Exception as e:
        if "429" in str(e) or "ResourceExhausted" in str(e):
            print("Cuota agotada. Esperando 60s...")
            time.sleep(60)
            return generate_answer(system_prompt)
        raise e

def ask_video(question):
    print(f"\n Tú: {question}")
    
    # PASO 1: RETRIEVAL
    try:
        q_emb = genai.embed_content(
            model="models/text-embedding-004",
            content=question,
            task_type="retrieval_query"
        )['embedding']
        
        results = collection.query(
            query_embeddings=[q_emb],
            n_results=3
        )
        
        # Preparar Contexto con Tiempos
        context_parts = []
        print("\n [Fuentes]:")
        for i, doc in enumerate(results['documents'][0]):
            ts = results['metadatas'][0][i]['timestamp']
            print(f"-- [{ts}] {doc[:80]}...")
            context_parts.append(f"({ts}) {doc}")

        # PASO 2: PROMPT CON HISTORIA
        # FIX: Calculamos la string fuera del f-string para evitar SyntaxError por backslash
        history_text = "\n".join([f"{m['role']}: {m['content']}" for m in chat_history[-3:]])
        context_joined = "\n".join(context_parts)
        
        prompt = f"""
        Eres un asistente experto. Responde basándote en el contexto.
        HISTORIAL:
        {history_text}
        CONTEXTO:
        {context_joined}
        PREGUNTA: {question}
        INSTRUCCIÓN: 
        1. Tienes MEMORIA: usa el historial para entender preguntas como "¿Y qué más?" o referencias a lo anterior.
        2. DEBES CITAR: Al final de cada afirmación importante, pon el timestamp entre corchetes. Ejemplo: "Según el video, las neuronas se activan [05:32]."
        3. Si no sabes la respuesta, dilo.
        """
        
        # PASO 3: GENERACIÓN
        answer = generate_answer(prompt)
        print(f"\n IA: {answer}")
        
        chat_history.append({"role": "Usuario", "content": question})
        chat_history.append({"role": "IA", "content": answer})

    except Exception as e:
        print(f"Error: {e}")

# BUCLE
print("\n" + "="*40)
print("¡Chat listo! Escribe 'exit' para salir.")
print("="*40 + "\n")

while True:
    user_input = input("Tú: ")
    if user_input.lower() in ['exit', 'salir', 'quit']:
        print("¡Hasta luego!")
        break
    ask_video(user_input)